import torch, torch.nn as nn, torchvision.models as models
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False          # freeze all
model.fc = nn.Linear(model.fc.in_features, num_classes)
# now train only model.fc parameters with higher LR


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def preprocess(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# assume datasets in Hugging Face datasets format: train/val/test
tokenized_train = train_dataset.map(preprocess, batched=True)
training_args = TrainingArguments(output_dir="out", per_device_train_batch_size=16, 
                                  per_device_eval_batch_size=32, num_train_epochs=3,
                                  evaluation_strategy="epoch", learning_rate=2e-5, weight_decay=0.01)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_train, eval_dataset=tokenized_val)
trainer.train()
